# Create a viewer to edit SingleCellStatic objects

In [1]:
# import some common libraries
import os
import glob
import os.path
import numpy as np
import json, random, cv2
from cellpose import models
from cellpose.io import imread
import matplotlib.pyplot as plt

from tqdm import tqdm
from pathlib import Path
from skimage import measure
from PIL import Image, ImageSequence

# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core import SingleCellTrajectory, SingleCellStatic
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset


In [2]:
from livecellx.sample_data import tutorial_three_image_sys
from pathlib import Path
from livecellx.core.datasets import LiveCellImageDataset
import glob


In [3]:
import napari
napari.__version__

'0.4.18'

load EBSS data

In [4]:

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecellx.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)


In [5]:
# from livecellx.core import SingleCellStatic
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_scs_EBSS_starvation/single_cells.json",
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")

In [6]:
# dic_dataset, mask_dataset = tutorial_three_image_sys()


In [7]:
# from skimage.measure import regionprops
# from livecellx.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json",
#                                          dataset_dir="./datasets/test_data_STAV-A549/datasets")

## Load single cells from json file

In [10]:
# scs_path = "./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
scs_path = r".\test_annotations\XY10\single_cells.json"

single_cells = SingleCellStatic.load_single_cells_json(path=scs_path)

In [11]:
dic_dataset = single_cells[0].img_dataset

In [13]:
from livecellx.core.sc_seg_operator import ScSegOperator, create_sc_seg_napari_ui
# replace the path below with your downdloaded CSN checkpoint path
ckpt_path = r"C:\Users\13290\Documents\Single-cell\LiveCell\LiveCellTracker-dev\notebooks\csn_models\v11-01_epoch=229-best.ckpt"
ScSegOperator.load_default_csn_model(path=ckpt_path, cuda=True);

Lightning automatically upgraded your loaded checkpoint from v1.8.6 to v2.0.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file C:\Users\13290\Documents\Single-cell\LiveCell\LiveCellTracker-dev\notebooks\csn_models\v11-01_epoch=229-best.ckpt`
c:\Users\13290\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\13290\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date we

>>> Using BCE loss with logits loss
>>> Based on loss type, training output threshold:  0.5


## Create a viewer to edit SingleCellStatic objects

In [14]:
def enable_gui_qt():
    from IPython import get_ipython

    ipython = get_ipython()
    ipython.run_line_magic('gui', 'qt')


enable_gui_qt()

In [15]:
from livecellx.core.sct_operator import create_scs_edit_viewer
sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset)

In [16]:
sct_operator.remove_empty_contour_sct()

In [17]:
# sct_operator.shape_layer.selected_data = [0]
# sc_operator = sct_operator.edit_selected_sc()
# sct_operator.shape_layer.selected_data = [0]
# sc_operator = sct_operator.edit_selected_sc()

In [18]:
# sc_operator.csn_correct_seg_callback()
# sc_operator.save_seg_callback()
# sc_operator.filter_cells_by_size_callback(min_size=100, max_size=1000)

In [19]:
# len(sc_operator.shape_layer.selected_data), len(sc_operator.shape_layer.data)

In [20]:
# sc_operator.resample_contours_callback(40)
# sc_operator.restore_sc_contour_callback()
# sct_operator.clear_sc_opeartors()
# sct_operator.add_new_sc()

In [21]:
sct_operator.sc_operators = []

In [22]:
all_scs = sct_operator.get_all_scs()
len(all_scs)

423

In [23]:
sct_operator.clear_sc_opeartors()

In [24]:
sct_operator.edit_selected_sc()

|-----? More than one shape is selected. The first selected shape is used for editing.
>>> create sc layer done


ScSegOperator(sc=SingleCellStatic(id=6a070fa2-18c4-4447-b19d-c1128a7a1c6a, timeframe=1223, bbox=[1593.  573. 1667.  636.]), mode=0)

In [25]:
sct_operator.clear_sc_opeartors()

clearing sc operator:  ScSegOperator(sc=SingleCellStatic(id=6a070fa2-18c4-4447-b19d-c1128a7a1c6a, timeframe=1223, bbox=[1593.  573. 1667.  636.]), mode=0)


In [26]:
# sct_operator.viewer.layers.remove(sct_operator.sc_operators[0].shape_layer)

Update SctOperator's shape layer by an updated single cell

In [27]:
sct_operator.update_shape_layer_by_sc(single_cells[0])

<update shape layer by sc>
clearing selection...
<clear complete>
current shape layer shape properties:  Event
setting face color of selected shape...
<selection complete>
clearing selection...
<clear complete>
<update shape layer by sc complete>


Get all the single cells from the sct operators

In [28]:
len(sct_operator.get_all_scs()), len(set(sct_operator.get_all_scs()))

(423, 423)

Save annotated single cells

In [29]:
# sct_operator.restore_shapes_data()

In [30]:
# all_scs = sct_operator.get_all_scs()
# SingleCellStatic.write_single_cells_json(all_scs, path="./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json", \
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")

In [31]:
sct_operator.select_info

[]

Programmatically delete a sample

In [33]:
sct_operator.annotate_click_samples["mitosis"][-1]

{}

In [ ]:
# for sc in sct_operator.annotate_click_samples["mitosis"][-1]:
#     sc.meta = {}
# sct_operator.annotate_click_samples["mitosis"].pop(-1)

In [ ]:
# sct_operator.save_annotations(sample_out_dir="./test_annoations")

In [34]:
scs = sct_operator.get_all_scs()

In [35]:
sct_operator.annotate_click_samples

{}

In [ ]:
len(sct_operator.annotate_click_samples["mitosis"])

Track based on single cells

In [36]:
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)

all_scs = sct_operator.get_all_scs()

all_scs = [sc for sc in all_scs if sc.timeframe != 1]
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

C:\Users\13290\Documents\Single-cell\LiveCell\LiveCellTracker-dev\livecellx\track\sort_tracker.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  r = w / float(h)
C:\Users\13290\Documents\Single-cell\LiveCell\LiveCellTracker-dev\livecellx\track\sort_tracker.py:93: RuntimeWarning: invalid value encountered in multiply
  w = np.sqrt(x[2] * x[3])


Show subset of a sctc filtered by time_span

In [37]:
from livecellx.core.sct_operator import create_scts_operator_viewer, create_scs_edit_viewer

scts_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, time_span=(145, 155))

# # If you would like to start from sctc, you can use the following code
# scts_operator = create_scts_operator_viewer(sctc, img_dataset = dic_dataset, time_span=(1, 2))

|-----> A new SCTC object with size 423 is created by subsetting the original sctc with time span (145, 155)


c:\Users\13290\AppData\Local\Programs\Python\Python39\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
c:\Users\13290\AppData\Local\Programs\Python\Python39\lib\site-packages\napari\utils\colormaps\standardize_color.py:200: UserWarning: Given color input is empty. Converting input to a white color array.
  warnings.warn(


In [ ]:
# sctc.write_json(path="./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json")